In [1]:
import cv2
import os

data_path='E:/Vinit/DATA SCIENCE/Gender Classification/Datasets'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

{'man': 0, 'woman': 1}
['man', 'woman']
[0, 1]


In [2]:
img_size=32
data=[]
target=[]

facedata = "E:/Vinit/DATA SCIENCE/Gender Classification/haarcascade_frontalface_default.xml"
cascade = cv2.CascadeClassifier(facedata)


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        faces = cascade.detectMultiScale(img)
        try:
            for f in faces:
                x, y, w, h = [v for v in f]
                sub_face = img[y:y + h, x:x + w]
                gray=cv2.cvtColor(sub_face,cv2.COLOR_BGR2GRAY)
                resized=cv2.resize(gray,(img_size,img_size))
                data.append(resized)
                target.append(label_dict[category])
        except Exception as e:
            print('Exception:',e)

In [3]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

np.save('E:/Vinit/DATA SCIENCE/Gender Classification/training/data',data)
np.save('E:/Vinit/DATA SCIENCE/Gender Classification/training/target',new_target)

Using TensorFlow backend.


In [4]:
import numpy as np

data=np.load('E:/Vinit/DATA SCIENCE/Gender Classification/training/data.npy')
target=np.load('E:/Vinit/DATA SCIENCE/Gender Classification/training/target.npy')

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

noOfFilters=64
sizeOfFilter1=(3,3)
sizeOfFilter2=(3,3)
sizeOfPool=(2,2)
noOfNode=64


model=Sequential()
model.add((Conv2D(32, sizeOfFilter1, input_shape=data.shape[1:],activation='relu')))
model.add((Conv2D(32, sizeOfFilter1,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add((Conv2D(64, sizeOfFilter2,activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(noOfNode, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [6]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [9]:
checkpoint = ModelCheckpoint('E:/Vinit/DATA SCIENCE/Gender Classification/training/model-{epoch:03d}.model.hdf5',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=50,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 2s 117ms/step - loss: 0.3263 - accuracy: 0.8497 - val_loss: 0.4641 - val_accuracy: 0.8557
Epoch 2/50
13/13 [==============================] - 1s 109ms/step - loss: 0.2708 - accuracy: 0.9093 - val_loss: 0.4697 - val_accuracy: 0.8660
Epoch 3/50
13/13 [==============================] - 1s 107ms/step - loss: 0.2679 - accuracy: 0.8990 - val_loss: 0.4679 - val_accuracy: 0.8763
Epoch 4/50
13/13 [==============================] - 1s 105ms/step - loss: 0.2004 - accuracy: 0.9249 - val_loss: 0.5128 - val_accuracy: 0.8351
Epoch 5/50
13/13 [==============================] - 1s 105ms/step - loss: 0.2274 - accuracy: 0.9119 - val_loss: 0.4660 - val_accuracy: 0.8557
Epoch 6/50
13/13 [==============================] - 1s 103ms/step - loss: 0.1983 - accuracy: 0.9275 - val_loss: 0.4854 - val_accuracy: 0.8660
Epoch 7/50
13/13 [==============================] - 1s 110ms/step - loss: 0.1714 - accuracy: 0.9404 - val_loss: 0.4336 - val_accuracy: 0.8557
Epoch 

In [10]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [ ]:
model = load_model('E:/Vinit/DATA SCIENCE/Gender Classification/training/model-019.model.hdf5')

face_clsfr=cv2.CascadeClassifier('E:/Vinit/DATA SCIENCE/Gender Classification/haarcascade_frontalface_default.xml')

cap=cv2.VideoCapture(0)

labels_dict={0:'Male',1:'Female'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [ ]:
while(True):
    
    ret,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,3)
    
    for (x,y,w,h) in faces:
        
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(32,32))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,32,32,1))
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('Result',img)
    key=cv2.waitKey(1)
    
    if (key==27):
        break
    if ret==False:    
        break
        
cv2.destroyAllWindows()
cap.release()